In [2]:
from dask.distributed import Client

In [3]:
 scheduler_address = "tcp://10.105.157.106:80"

In [1]:
client = Client(scheduler_address)
print(client)

In [2]:
from datasets import load_dataset, DatasetDict, Dataset
import pandas as pd

dataset = load_dataset("Bingsu/Cat_and_Dog")

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
import torchvision.models as models
from torchvision import transforms
from PIL import Image
# import matplotlib.pyplot as plt

device = 'cpu'

model_resnet18 = torch.hub.load('pytorch/vision', 'resnet18', pretrained=True).to(device)

In [ ]:
from tqdm import tqdm
import time

def train(model, optimizer, loss_fn, train_loader, test_loader, epochs=5):

    # model = model.to(device)
    # model = model.share_memory()

    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()

        # Create a progress bar for the training loop
        with tqdm(enumerate(train_loader), total=len(train_loader), desc=f'Epoch {epoch + 1}/{epochs}', dynamic_ncols=True, unit='batch', unit_scale=True) as train_progress_bar:
            start_time = time.time()
            for batch_idx, batch in train_progress_bar:

                # batch = [b for b in batch]
                # futures = client.map(lambda data: model(data[0]), batch)

                # losses = client.map(loss_fn, futures, batch[1])
                # loss = client.submit(torch.mean, client.gather(losses))
                # # Compute the gradients and update the model parameters
                # optimizer.zero_grad()
                # gradients = client.map(lambda loss, future: torch.autograd.grad(loss, future)[0], loss, futures)
                # gradients =client.submit(torch.mean, client.gather(gradients))

                optimizer.zero_grad()
                inputs, targets = batch['image'], batch['labels']
                output = model(inputs)
                loss = loss_fn(output, targets)
                loss.backward()
                optimizer.step()
                training_loss += loss.item() * inputs.size(0)

                # Update the progress bar
                train_progress_bar.set_postfix({'Training Loss': training_loss})

        training_loss /= len(train_loader.dataset)

        print('Epoch: {}, Training Loss: {:.4f}'.format(epoch, training_loss))


    # client.map(lambda parameter, gradient: parameter.grad.copy_(gradient), model.parameters(), gradients)
    #optimizer.step()

            # model.eval()
        # num_correct = 0
        # num_examples = 0


        # # Create a progress bar for the testing loop
        # with tqdm(enumerate(test_loader), total=len(test_loader), desc='Testing', dynamic_ncols=True, unit='batch', unit_scale=True) as test_progress_bar:
        #     for batch_idx, batch in test_progress_bar:
        #         inputs, targets = batch['image'], batch['labels']
        #         output = model(inputs)
        #         loss = loss_fn(output, targets)
        #         valid_loss += loss.item() * inputs.size(0)

        #         correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets).view(-1)
        #         num_correct += torch.sum(correct).item()
        #         num_examples += correct.shape[0]

        # valid_loss /= len(test_loader.dataset)

        # # Display the training and testing results with accuracy
        # print('Epoch: {}, Training Loss: {:.4f}, Testing Loss: {:.4f}, accuracy_test = {:.4f}'.format(epoch, training_loss,
        #                                                                                               valid_loss, num_correct / num_examples))


In [ ]:
batch_size=32
img_dimensions = 224


img_transforms = transforms.Compose([
    transforms.Resize((img_dimensions, img_dimensions)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225] )
    ])

img_test_transforms = transforms.Compose([
    transforms.Resize((img_dimensions,img_dimensions)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225] )
    ])

def transforms_h (data):
  data['image'] = [img_transforms(i) for i in data['image']]
  return data

def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False


num_workers = 6


dataset.set_transform(transforms_h)

train_data_loader = torch.utils.data.DataLoader(dataset['train'], batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_data_loader = torch.utils.data.DataLoader(dataset['test'], batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [ ]:
print(f'Num training images: {len(train_data_loader.dataset)}')
print(f'Num test images: {len(test_data_loader.dataset)}')

In [4]:
def test_model(model, test_data_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_data_loader:
            images, labels = data['image'].to(device), data['labels'].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('correct: {:d}  total: {:d}'.format(correct, total))
    print('accuracy = {:f}'.format(correct / total))

In [ ]:
optimizer = optim.Adam(model_resnet18.parameters(), lr=0.0001)
train(model_resnet18, optimizer, torch.nn.CrossEntropyLoss(), train_data_loader, test_data_loader, epochs=5)

In [5]:
test_model(model_resnet18, test_data_loader)